# Fine-tuning

We create a fine-tuning job on the Together AI platform for Qwen2.5-32B-Instruct.

In [6]:
from dotenv import load_dotenv; load_dotenv()
import random as random
import openai
import os 
import time
from pathlib import Path
from together import Together


api_key = os.getenv("VALUESLAB_OPENAI")
client = openai.OpenAI(api_key=api_key)
fireworks_api_key = os.getenv("FIREWORKS_API_KEY")
fireworks_account_id = os.getenv("FIREWORKS_ACCOUNT_ID")
together_api_key = os.getenv("TOGETHER_API_KEY") 

In [7]:
# ============== CONFIGURATION ==============
EPOCHS = 4
LEARNING_RATE = 2e-4
LORA_RANK = 32
BASE_MODEL = "Qwen/Qwen2.5-32B-Instruct"

together_client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

runs = [
    {"name": "southern", "path": Path("data/southern.jsonl")},
    {"name": "coastal", "path": Path("data/coastal.jsonl")}
]

# ============== UPLOAD & FINE-TUNE ==============
for run in runs:
    name, path = run["name"], run["path"]
    created = {}

    ts = time.strftime("%Y%m%d-%H%M%S")
    print(f"\n=== {name.upper()} ===")

    # 1) Upload file
    print(f"Uploading {path}...")
    file_resp = together_client.files.upload(file=path)
    file_id = file_resp.id
    print(f"File uploaded: {file_id}")

    # 2) Wait for file processing
    print("Waiting for file processing...")
    while True:
        file_status = together_client.files.retrieve(file_id)
        print(f"  processed: {file_status.processed}")
        if file_status.processed:
            break
        time.sleep(5)

    # 3) Create fine-tuning job
    print("Starting fine-tune job...")
    job = together_client.fine_tuning.create(
        training_file=file_id,
        model=BASE_MODEL,
        n_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        lora_r=LORA_RANK,
        lora=True,
        suffix=f"{name}-{ts}"
    )
    print(f"Job created: {job.id}")

    created[name] = {
        "job_id": job.id,
        "file_id": file_id,
        "timestamp": ts
    }

    print("\n" + "="*50)
    print("Jobs launched!")
    for name, info in created.items():
        print(f"  {name}: {info['job_id']}")
    print("\nCheck status at: https://api.together.xyz/jobs")


=== SOUTHERN ===
Uploading data/southern.jsonl...


Validating file: 99 lines [00:00, 60742.55 lines/s]
Uploading file southern.jsonl: 100%|██████████| 13.5k/13.5k [00:00<00:00, 26.3kB/s]


File uploaded: file-4c6a1d65-d2cd-4bb2-a99c-178bb6203263
Waiting for file processing...
  processed: True
Starting fine-tune job...
Job created: ft-67c08f55-7baa

Jobs launched!
  southern: ft-67c08f55-7baa

Check status at: https://api.together.xyz/jobs

=== COASTAL ===
Uploading data/coastal.jsonl...


Validating file: 98 lines [00:00, 55366.62 lines/s]
Uploading file coastal.jsonl: 100%|██████████| 13.4k/13.4k [00:00<00:00, 42.5kB/s]


File uploaded: file-e1aa2ca2-1210-4330-b1ad-04c304695891
Waiting for file processing...
  processed: True
Starting fine-tune job...
Job created: ft-64380700-ce97

Jobs launched!
  coastal: ft-64380700-ce97

Check status at: https://api.together.xyz/jobs
